# Which activation function to use?
#### Author: JP Melo

### Imports

In [1]:
from derpinns.nn import *
from derpinns.utils import *
from derpinns.trainer import *
import torch.nn as nn
import torch

## Parameters

In [ ]:
# Fix seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Global parameters
assets = 3

sampler = "pseudo"
nn_shape = "10x3"
device = torch.device("cpu")
dtype = torch.float32

# Define option valuation params
params = OptionParameters(
    n_assets=assets,
    tau=1.0,
    sigma=np.array([0.2] * assets),
    rho=np.eye(assets) + 0.25 * (np.ones((assets, assets)) - np.eye(assets)),
    r=0.05,
    strike=100,
    payoff=payoff
)

boundary_samples = 100
interior_samples = boundary_samples*assets*2
initial_samples = boundary_samples*assets*2

dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)
    
batch_size = len(dataset)  # we use all samples


## Training

### Tanh

In [ ]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32,
    activation=nn.Tanh(),
).apply(weights_init).to(device)

model.train()

optimizer = SSBroyden(
    model.parameters(),
    max_iter=1000,
)

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)

trainer.train()

SSBroyden training:  98%|█████████▊| 984/1000 [06:36<00:06,  2.48it/s, Interior=0.000191, Boundary=0.000205, Initial=0.000555, Total=0.000952, Max Error=16.316452, L2 Error=0.020818] 


In [4]:
with_tanh = trainer.closure.get_state()
plot_loss(with_tanh, smooth=False)

tanh_l2 = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", tanh_l2*100)

L2 Error:  1.4283074


### ReLU

In [ ]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32,
    activation=nn.ReLU(),
).apply(weights_init).to(device)

model.train()

optimizer = SSBroyden(
    model.parameters(),
    max_iter=1000,
)

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)

trainer.train()

SSBroyden training:  29%|██▉       | 147/500 [00:59<02:22,  2.48it/s, Interior=0.001217, Boundary=0.004048, Initial=0.005036, Total=0.010302, Max Error=17.764824, L2 Error=0.039665]


In [6]:
with_relu = trainer.closure.get_state()
plot_loss(with_relu, smooth=False)

relu_l2 = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", relu_l2*100)

L2 Error:  4.6840525


### SoftPlus

In [12]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32,
    activation=nn.Softplus(),
).apply(weights_init).to(device)

model.train()

optimizer = SSBroyden(
    model.parameters(),
    max_iter=1000,
)

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)

trainer.train()

SSBroyden training: 100%|██████████| 1000/1000 [06:38<00:00,  2.51it/s, Interior=0.000091, Boundary=0.000107, Initial=0.000222, Total=0.000420, Max Error=2.748550, L2 Error=0.004663]


In [13]:
with_softplus = trainer.closure.get_state()
plot_loss(with_softplus, smooth=False)

softplus_l2 = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", softplus_l2*100)

L2 Error:  1.3188713


### ELU

In [14]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32,
    activation=nn.ELU(),
).apply(weights_init).to(device)

model.train()

optimizer = SSBroyden(
    model.parameters(),
    max_iter=1000,
)

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)

trainer.train()

SSBroyden training:   7%|▋         | 74/1000 [00:30<06:23,  2.42it/s, Interior=0.002693, Boundary=0.007040, Initial=0.015566, Total=0.025299, Max Error=34.213135, L2 Error=0.046047]  


In [15]:
with_elu = trainer.closure.get_state()
plot_loss(with_elu, smooth=False)

elu_l2 = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", elu_l2*100)

L2 Error:  5.0847144


### Compare runs

In [17]:
compare_loss_histories(
    [with_tanh, with_relu, with_softplus, with_elu],
    labels=["Tanh", "ReLU", "Softplus", "ELU"],
    smooth=False,
)